In [1]:
import torch
from PerceiverAR import *
from dataset import *
from trainer import *

In [2]:
BLOCK_SIZE = 128
LATENT_SIZE = 32
CORPUS_PATH = "wiki.txt"

In [3]:
text = open(CORPUS_PATH, encoding="utf-8").read()
perceiver_dataset = PerceiverARDataset(text, BLOCK_SIZE, LATENT_SIZE)

data has 418352 characters, 258 unique.


In [4]:
model_config = GPTConfig(
    perceiver_dataset.vocab_size,
    perceiver_dataset.block_size,
    n_layer=4,
    n_head=8,
    n_embd=256,
    perceiver=True,
    latent_size=perceiver_dataset.latent_size,
    padding_idx=perceiver_dataset.stoi[perceiver_dataset.PAD_CHAR],
)

model = PerceiverAR(model_config)

# Train

In [10]:
trainer_config = TrainerConfig(
    max_epochs=1,
    lr_decay=True,
    ckpt_path="perceiver_ar.pt",
)
model.load_state_dict(torch.load(trainer_config.ckpt_path))
trainer = Trainer(model, perceiver_dataset, trainer_config)

In [ ]:
trainer.train()

# Test

In [11]:
TEST_INDEX = 200

In [12]:
def torch_to_string(x):
    x = x.to("cpu")
    return "".join([perceiver_dataset.itos[i] for i in x.numpy()])

In [13]:
x, y = perceiver_dataset[TEST_INDEX]
x, y = x.to(trainer.device), y.to(trainer.device)
model.eval()
y_pred = model(x.unsqueeze(0))
max_indices = y_pred.argmax(dim=2)

In [14]:
x_string = torch_to_string(x)[:-LATENT_SIZE+1]
y_string = torch_to_string(y)
y_pred_string = torch_to_string(max_indices.squeeze(0))

In [15]:
print(f"Start - {x_string}")
print(f"Target - {y_string}")
print(f"Predicted - {y_pred_string}")

Start - □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□◎Douglas Fee. Douglas Fee (born 21 July 1944 in Kingston, On
Target - tario) was a member of the Canad
Predicted -     nn i   i i   o  i  i   i    


In [ ]:
# from torch.utils.data import DataLoader
# batch_size = 32
# data_loader = DataLoader(perceiver_dataset, batch_size=batch_size, shuffle=True)
# data_iter = iter(data_loader)
# x, y = next(data_iter)